### 텐서플로우를 이용하여 NN구축하기


In [2]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

tf.compat.v1.disable_v2_behavior()
# Loading the data set
raw_data = load_iris()

# split data set
X_train, X_test, Y_train, Y_test = train_test_split(
    raw_data.data, raw_data.target, test_size=0.33, random_state=42, stratify=raw_data.target)

# max min scaling
X_scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.fit_transform(X_test)

# One hot encode Y
onehot_encoder = OneHotEncoder(sparse=False)
Y_train_enc = onehot_encoder.fit_transform(Y_train.reshape(-1, 1))
Y_test_enc = onehot_encoder.fit_transform(Y_test.reshape(-1, 1))

# Define Model Parameters
learning_rate = 0.01
training_epochs = 10000

# define the number of neurons
layer_1_nodes = 150
layer_2_nodes = 150

# define the number of inputs
num_inputs = X_train_scaled.shape[1]
num_output = len(np.unique(Y_train, axis=0))

# Define the layers
with tf.compat.v1.variable_scope('input'):
    X = tf.compat.v1.placeholder(tf.float32, shape=(None, num_inputs))

with tf.compat.v1.variable_scope('layer_1'):
    weights = tf.compat.v1.get_variable('weights1', shape=[num_inputs, layer_1_nodes],
                                        initializer=tf.keras.initializers.glorot_normal())
    biases = tf.compat.v1.get_variable(
        'bias1', shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

with tf.compat.v1.variable_scope('layer_2'):
    weights = tf.compat.v1.get_variable('weights2', shape=[layer_1_nodes, layer_2_nodes],
                                        initializer=tf.keras.initializers.glorot_normal())
    biases = tf.compat.v1.get_variable(
        'bias2', shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

with tf.compat.v1.variable_scope('output'):
    weights = tf.compat.v1.get_variable('weights3', shape=[layer_2_nodes, num_output],
                                        initializer=tf.keras.initializers.glorot_normal())
    biases = tf.compat.v1.get_variable(
        'bias3', shape=[num_output], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_2_output, weights) + biases

with tf.compat.v1.variable_scope('cost'):
    # use 1 instead of num output unless one hot encoding??
    Y = tf.compat.v1.placeholder(tf.float32, shape=(None, num_output))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=prediction))

with tf.compat.v1.variable_scope('train'):
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(
        learning_rate).minimize(cost)

with tf.compat.v1.variable_scope('accuracy'):
    correct_prediction = tf.equal(
        tf.argmax(Y, axis=1), tf.argmax(prediction, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize a session so that we can run TensorFlow operations
saver = tf.compat.v1.train.Saver()

with tf.compat.v1.Session() as session:
    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.compat.v1.global_variables_initializer())

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_train_scaled, Y: Y_train_enc})

        # Every 5 training steps, log our progress
        if epoch % 50 == 0:
            training_cost = session.run(
                [cost], feed_dict={X: X_train_scaled, Y: Y_train_enc})
            testing_cost = session.run(
                [cost], feed_dict={X: X_test_scaled, Y: Y_test_enc})

            # accuracy
            train_accuracy = session.run(
                accuracy, feed_dict={X: X_train_scaled, Y: Y_train_enc})
            test_accuracy = session.run(
                accuracy, feed_dict={X: X_test_scaled, Y: Y_test_enc})

            print(epoch, training_cost, testing_cost,
                  train_accuracy, test_accuracy)

    print("Training is complete!\n")
    final_train_accuracy = session.run(
        accuracy, feed_dict={X: X_train_scaled, Y: Y_train_enc})
    final_test_accuracy = session.run(
        accuracy, feed_dict={X: X_test_scaled, Y: Y_test_enc})
    print("Final Training Accuracy: {}".format(final_train_accuracy))
    print("Final Testing Accuracy: {}".format(final_test_accuracy))
    saver.save(session, "./my_model_final.ckpt")


Init Plugin
Init Graph Optimizer
Init Kernel
Instructions for updating:
non-resource variables are not supported in the long term
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

0 [1.0830811] [1.0839505] 0.32999998 0.34


2021-10-15 09:37:49.496059: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-10-15 09:37:49.496163: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-10-15 09:37:49.499240: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-10-15 09:37:49.499301: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-15 09:37:49.508763: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-15 09:37:49.571156: I tensorflow/core/grappler/o

50 [0.99753857] [0.9997004] 0.59 0.58
100 [0.9287085] [0.93282235] 0.66999996 0.65999997
150 [0.86410177] [0.8701706] 0.66999996 0.68
200 [0.80066764] [0.808565] 0.69 0.7
250 [0.7391118] [0.74835503] 0.71 0.7
300 [0.6810601] [0.6916959] 0.71999997 0.7
350 [0.6280363] [0.64002496] 0.74 0.74
400 [0.5805448] [0.5938381] 0.79999995 0.74
450 [0.5390874] [0.55347204] 0.82 0.82
500 [0.50339055] [0.5187094] 0.84 0.84
550 [0.4724694] [0.48864615] 0.85999995 0.84
600 [0.44586173] [0.46266207] 0.88 0.88
650 [0.42248285] [0.43992057] 0.90999997 0.88
700 [0.40168074] [0.4196675] 0.90999997 0.9
750 [0.3828711] [0.40115264] 0.90999997 0.96
800 [0.36542106] [0.3839276] 0.91999996 0.96
850 [0.34914008] [0.36783758] 0.94 0.96
900 [0.33379793] [0.35270935] 0.95 0.97999996
950 [0.31916842] [0.33830804] 0.95 0.97999996
1000 [0.3051659] [0.32455063] 0.95 0.97999996
1050 [0.291744] [0.31139126] 0.96 0.97999996
1100 [0.27886543] [0.29882452] 0.96999997 0.97999996
1150 [0.26655075] [0.28685537] 0.96999997 0.97

2021-10-15 09:38:49.792148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


- 학습된 모델 사용하기

In [4]:
with tf.compat.v1.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    while (True):
        sepal_length = input("sepal length (cm) ? ")
        sepal_width = input("sepal width (cm) ? ")
        petal_length = input("petal length (cm) ? ")
        petal_width = input("petal width (cm) ? ")
        new_data = [[sepal_length, sepal_width, petal_length, petal_width]]
        scaled_data = X_scaler.transform(new_data)
        predicted = prediction.eval(feed_dict={X: scaled_data})
        cls = np.argmax(predicted, axis=1)
        print(cls[0])


INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


2021-10-15 09:39:09.277231: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-10-15 09:39:09.277269: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-10-15 09:39:09.283407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-10-15 09:39:20.375155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


2
2


ValueError: could not convert string to float: ''